In [0]:
pip install faker azure-eventhub

In [0]:
import json
import random
import time
from datetime import datetime, timedelta
from faker import Faker
from azure.eventhub import EventHubProducerClient, EventData

# Initialize Faker
fake = Faker()

# Azure Event Hub Connection (replace with Key Vault secret retrieval in real projects)
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-device-policy")
EVENTHUB_NAME = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-device-hub")

# Device and Customer Info
CUSTOMER_IDS = list(range(1000, 1101))
DEVICE_BRANDS = {
    "Samsung": ["Galaxy S21", "Galaxy S22", "Galaxy S23"],
    "Apple": ["iPhone 13", "iPhone 14", "iPhone 15"],
    "OnePlus": ["Nord CE", "OnePlus 9", "OnePlus 11"],
    "Xiaomi": ["Redmi Note 11", "Mi 11X", "Poco F4"],
    "Google": ["Pixel 6", "Pixel 7", "Pixel 8"],
    "Huawei": ["P40", "P50", "Mate 40"]
}
OS_MAP = {
    "Samsung": "Android 14",
    "Apple": "iOS 18",
    "OnePlus": "Android 14",
    "Xiaomi": "Android 13",
    "Google": "Android 14",
    "Huawei": "HarmonyOS 4"
}
NETWORK_TYPES = ["4G", "5G"]

# Function to generate device record
def generate_device_record(device_id,customer_id):
    customer_id = f"{customer_id+1000:04d}"
    brand = random.choice(list(DEVICE_BRANDS.keys()))
    model = random.choice(DEVICE_BRANDS[brand])
    return {
        "device_id": f"DVC{device_id}",
        "customer": customer_id,
        "imei_number": str(random.randint(100000000000000, 999999999999999)),
        "devicebrand": brand,
        "model": model,
        "os": OS_MAP[brand],
        "sim_number": str(random.randint(8991101200000000000, 8991101299999999999)),
        "imsi": str(random.randint(405870000000000, 405879999999999)),
        "network_type": random.choice(NETWORK_TYPES),
        "activation_date": (datetime.now() - timedelta(days=random.randint(30, 365))).strftime("%Y-%m-%d"),
        "status": random.choice(["Active", "Suspended", "Inactive"]),
        "last_synced": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

# Event Hub Producer
producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STR, eventhub_name=EVENTHUB_NAME)

with producer:
    for i in range(50):  # generate and send 10 records
        record = generate_device_record(i+1,i+1)
        event_data = EventData(json.dumps(record))
        producer.send_batch([event_data])
        print(f"✅ Sent: {record}")
        time.sleep(1)  # wait 1 sec between events